written for https://stackoverflow.com/q/49888741/425458

In [1]:
from mpmath import mp, mpf
import numpy as np

# stands for "decimal places". Larger values 
# mean higher precision, but slower computation
mp.dps = 75

def tompf(arr):
    """Convert any numpy array to one of arbitrary precision mpmath.mpf floats
    """
    if arr.size and not isinstance(arr.flat[0], mpf):
        return np.array([mpf(x) for x in arr.flat]).reshape(*arr.shape)
    else:
        return arr

def dotmpf(arr0, arr1):
    """An arbitrary precision version of np.dot
    """
    return tompf(arr0).dot(tompf(arr1))

In [60]:
dim = 8
bshape = (dim,)*4
dshape = (dim,)*2

B = np.random.rand(*bshape)
BT = np.swapaxes(B, -2, -1)

d = np.random.rand(*dshape)
D = d.dot(d.T)

In [68]:
M = np.dot(np.dot(B, D), BT)
print('dot', np.sum(M - M.T))

import functools
M = functools.reduce(np.dot, [B,D,BT])
print('reduce->dot', np.sum(M - M.T))

# np.einsum('ij,jk,lk',B,D,B)
M = np.einsum('abcd,de,efgh',B,D,BT)
print('einsum', np.sum(M - M.T))

# np.einsum('ij,jk,lk',B,D,B)
M = np.einsum('abci,ij,jkml',B,D,B)
print('einsum w/transpose', np.sum(M - M.T))

dot 1.81898940355e-11
reduce->dot 1.81898940355e-11
einsum -7.27595761418e-12
einsum w/transpose -7.27595761418e-12


In [69]:
# np.einsum('ij,jk,lk',B,D,B)
M = np.einsum('abci,ij,jdfe',B,D,B)
print('einsum w/transpose', np.sum(M - M.T))

einsum w/transpose -7.27595761418e-12


In [64]:
np.allclose(BT, np.einsum('jdfe', B))

False

In [57]:
np.allclose(BT, np.einsum('wxzy', B))

True

In [54]:
np.allclose(np.swapaxes(B, -2, -1), np.swapaxes(B, 2, 3))

True

0.0

-4.8885340220294893e-12

In [21]:
M.shape

(4, 4, 4, 4, 4, 4)

In [6]:
# np.einsum('ij,jk,lk',B,D,B)
M = np.einsum('ab,bc,cd',B,D,B.T)
np.sum(M - M.T)

0.0

In [35]:
M = dotmpf(dotmpf(B, D), BT)
np.sum(M - M.T)

mpf('0.0')

<br/><br/><br/><br/><br/><br/><br/>
# Float investigation

In [85]:
print((0.84e-18 + 0.96e-18 + 0.75e-18) - (0.75e-18 + 0.84e-18 + 0.96e-18))

x = 0.84e-18 + 0.96e-18 + 0.75e-18
y = 0.75e-18 + 0.84e-18 + 0.96e-18
print(x - y)

-3.851859888774472e-34
-3.851859888774472e-34


In [85]:
print((0.84e-18 + 0.96e-18 + 0.75e-18) - (0.75e-18 + 0.84e-18 + 0.96e-18))

x = 0.84e-18 + 0.96e-18 + 0.75e-18
y = 0.75e-18 + 0.84e-18 + 0.96e-18
print(x - y)

-3.851859888774472e-34
-3.851859888774472e-34


In [ ]:
a = 0.84e-18
b = 0.96e-18
c = 0.75e-18

x = a + b + c
y = c + a + b

w=0; w+=a; w+=b; w+=c
z=0; z+=c; z+=a; z+=b

print(x - y)
print(x - w)
print(x - z)
print(w - z)

In [96]:
a = 0.84660776e-14  
b = 0.96026596e-14  
c = 0.74865727e-14

print((a+b) - (b+a))
print((a+b+c) - (c+a+b))

print((a*b) - (b*a))
print((a*b*c) - (c*a*b))

0.0
-3.1554436208840472e-30
0.0
0.0


In [164]:
A = np.random.rand(int(1e6))
AP = np.roll(A, 1)

B = np.random.rand(int(1e6))
BP = np.roll(B, 1)

print((A[0] + A[1]) - (A[1] + A[0]))
print((A[0] + A[1] + A[2]) - (A[2] + A[0] + A[1]))

print((A[0]*A[1]) - (A[1]*A[0]))
print((A[0]*A[1]*A[2]) - (A[2]*A[0]*A[1]))

print(np.sum(B) - np.sum(BP))
print(np.prod(B) - np.prod(BP))

0.0
0.0
0.0
0.0
-2.91038304567e-10
0.0


In [200]:
A = np.random.rand(int(1e7))
AP = np.roll(A, 1)

print(np.sum(A) - np.sum(AP))

-9.31322574615e-10


In [250]:
A = np.random.rand(int(1e2))
B = np.random.rand(int(1e2))
C = np.random.rand(int(1e2))

print(np.sum(A*B*C - C*B*A))
print(np.sum(A*B*C - C*B*A) - np.sum((A*B)*C - A*(B*C)))

-7.02630770406e-17
0.0


In [400]:
A = np.random.rand(int(1e2))
B = np.random.rand(int(1e2))
C = np.random.rand(int(1e2))

In [401]:
x = A[0]*B[0]*C[0]
y = B[0]*A[0]*C[0]
z = A[0]*C[0]*B[0]

w = C[0]*B[0]*A[0]
v = A[0]*(B[0]*C[0])

print(np.sum(x - y))
print(np.sum(x - z))
print(np.sum(y - z))

print(np.sum(x - w))
print(np.sum(x - v))

print(np.sum(x - w) - np.sum(x - v))

0.0
-3.46944695195e-18
-3.46944695195e-18
-3.46944695195e-18
-3.46944695195e-18
0.0


In [ ]:
A = np.random.rand(int(1e2))
B = np.random.rand(int(1e2))

In [406]:
X = A*B
Y = B*A

print(np.sum(X - Y))

0.0


In [424]:
A = np.random.rand(int(1e2))
B = np.random.rand(int(1e2))
C = np.random.rand(int(1e2))

In [425]:
X = A*B*C
Y = B*A*C
Z = C*(A*B)

W = C*B*A
V = A*(B*C)

print(np.sum(X - Y))
print(np.sum(X - Z))
print(np.sum(Y - Z))

print(np.sum(X - W))
print(np.sum(X - V))

print(np.sum(X - W) - np.sum(X - V))

0.0
0.0
0.0
-8.93382590128e-17
-8.93382590128e-17
0.0


In [405]:
a = 0.28932505388351715
b = 0.35486670150988275
c = 0.36691426775649227

x = a*b*c
y = b*a*c
z = a*c*b

w = c*b*a
v = a*(b*c)

# transposing a and b works as expected
print(x - y) # == 0

# transposing b and c seems to break the assumption of commutativity
print(x - z) # != 0
print(y - z) # != 0

# cyclic permutation doesn't work
print(x - w) # != 0

# associativty doesn't work
print(x - v) # != 0

# cyclic permutation and different associativity are wrong in the same way
print((x - w) - (x - v)) # == 0

0.0
6.938893903907228e-18
6.938893903907228e-18
6.938893903907228e-18
6.938893903907228e-18
0.0


In [457]:
a = 0.28932505388351715
b = 0.35486670150988275
c = 0.36691426775649227

# construct a bunch of vars that "should" all be equal 
# ie x==y==z==w

x = a*b*c
y = b*a*c   # x==y by commutativity (swap a and b)
z = c*(b*a) # y==z by commutativity (swap (b*a) and c)
w = c*b*a   # z==w by associativity (move the parenthesis from (b*a) to (c*b)); this isn't true for float arithmetic

xstr = 'a*b*c'
ystr = 'b*a*c'
zstr = 'c*(b*a)'
wstr = 'c*b*a'

print('commutativity holds in floating point arithmetic')
print("%s - %s ==" % (xstr, ystr), x - y, end='\n\n')
print("%s - %s ==" % (xstr, zstr), x - z, end='\n\n')
print("%s - %s ==" % (ystr, zstr), y - z, end='\n\n')

print('associativity does not')
print("%s - %s ==" % (xstr, wstr), x - w, end='\n\n')

commutativity holds in floating point arithmetic
a*b*c - b*a*c == 0.0

a*b*c - c*(b*a) == 0.0

b*a*c - c*(b*a) == 0.0

associativity does not
a*b*c - c*b*a == 6.938893903907228e-18



# How to use `numpy.einsum`

In [446]:
np.dot(np.dot(d, D), d.T)

array([[ 0.49389954,  0.86786482,  0.66339628],
       [ 0.86786482,  1.52792247,  1.16644426],
       [ 0.66339628,  1.16644426,  0.89251433]])

In [447]:
np.dot(np.dot(d.T, D), d)

array([[ 0.15368338,  0.45268695,  0.47464036],
       [ 0.45268695,  1.3666801 ,  1.46473265],
       [ 0.47464036,  1.46473265,  1.59980095]])

In [463]:
np.einsum('ij,jk,lk',d,D,d)

array([[ 0.49389954,  0.86786482,  0.66339628],
       [ 0.86786482,  1.52792247,  1.16644426],
       [ 0.66339628,  1.16644426,  0.89251433]])

In [468]:
np.einsum('cd,de,fe',d,D,d)

array([[ 0.49389954,  0.86786482,  0.66339628],
       [ 0.86786482,  1.52792247,  1.16644426],
       [ 0.66339628,  1.16644426,  0.89251433]])

In [469]:
np.einsum('cd,de,ef',d,D,d.T)

array([[ 0.49389954,  0.86786482,  0.66339628],
       [ 0.86786482,  1.52792247,  1.16644426],
       [ 0.66339628,  1.16644426,  0.89251433]])

In [467]:
np.einsum(d,[0,1],D,[1,2],d,[3,2])

array([[ 0.49389954,  0.86786482,  0.66339628],
       [ 0.86786482,  1.52792247,  1.16644426],
       [ 0.66339628,  1.16644426,  0.89251433]])